# Formating tables for publications
This has to be done to make the tables understandable. In hindsight it would have been better to use a library like `TableOne`.

First we import the needed library:

In [1]:
from pathlib import Path
import re

import pandas as pd
import numpy as np

## Logit / Frequency tables

Now we will define functions and helper variables for formatting:

In [2]:
# Variables will be renamed by the values of this dict and sorted to match the order of the keys 
variable_rename_and_reorder_map = {
    # Requirements
    'risk_management_plan': 'Study required by a Risk Management Plan',
    'requested_by_regulator': 'Study required by a regulatory body',
    # Funding
    'funding_sources_grouped': 'Type of funding source',
    'multiple_funding_sources': 'Multiple funding sources',
    'number_of_studies_funded_by_biggest_sponsor_quartiles': 'Number of PAS funded by sponsor, quartiles',
    # Study
    'number_of_countries_grouped': 'Countries in which study is conducted',
    'study_type': 'Study type',
    'number_of_subjects_grouped': 'Estimated study population size',
    'age_population': 'Age of study population',
    'studied_medical_conditions': 'Medical condition(s) to be studied',
    'has_outcomes': 'Outcomes specified',
    # Research network and Data sources
    'collaboration_with_research_network': 'Collaboration of a research network',
    'uses_established_data_source': 'Study uses established data source',
    # Date based
    'updated_state': 'Status of study',
    'planned_duration_quartiles': 'Planned duration of study, quartiles',
    # Outcomes
    'has_protocol': 'Study made protocol public',
    'has_result': 'Study made final report public'
}

In [3]:
value_rename_map = {
    'True': 'Yes',
    'False': 'No',
    '18+ years': '≥18 years',
    '18+ years; <18 years': '<18 and ≥18 years',
    'Not available': 'Unspecified',
    'EU RMP category 1 (imposed as condition of marketing authorisation)': 'EU RMP category 1',
    'EU RMP category 2 (specific obligation of marketing authorisation)': 'EU RMP category 2',
    'EU RMP category 3 (required)': 'EU RMP category 3',
    'Non-interventional study': 'Non-interventional'
}

In [8]:
INPUT_FOLDER = Path('../../output/ema_rwd')

quartile_intervals = {
    file.name: np.load(file)
    for file in INPUT_FOLDER.glob('*_quartile_intervals*.npy')
}

for [k, v] in sorted(quartile_intervals.items()):
    display(k, (np.round((v.astype('timedelta64[D]').astype(float) / 365.25), 2).tolist(), v.astype('timedelta64[D]').astype(int).tolist()) if v.dtype == 'timedelta64[ns]' else v.tolist())

'number_of_studies_funded_by_biggest_sponsor_quartile_intervals_all.npy'

[1.0, 8.0, 58.0, 110.0, 176.0]

'number_of_studies_funded_by_biggest_sponsor_quartile_intervals_due_protocol.npy'

[1.0, 9.0, 58.0, 110.0, 176.0]

'number_of_studies_funded_by_biggest_sponsor_quartile_intervals_due_result.npy'

...

([0.02, 0.83, 2.08, 4.57, 35.58], [6, 304, 761, 1670, 12995])

'planned_duration_quartile_intervals_due_protocol.npy'

([0.02, 0.84, 2.17, 4.66, 23.03], [6, 305, 791, 1701, 8412])

'planned_duration_quartile_intervals_due_result.npy'

([0.02, 0.67, 1.64, 3.35, 19.74], [6, 243, 598, 1224, 7211])

In [8]:
def rename_and_reorder(df: pd.DataFrame, type='frequency', sort_references_to_start=True, multivariate_only=False):

    pre_order = df.rename(index=value_rename_map, level=1)
    
    if type == 'frequency':
        pre_order = (
            pre_order
            .rename_axis(index=['Variable, n (%)', 'Category'])
            .rename(columns=lambda x : x.replace('(', '(N='))
            .drop('registration_year_grouped', level=0)
        )
    elif type == 'logit':
        pre_order = (
            pre_order
            .rename_axis(index=['Variable', 'Category'])
            .drop(['registration_year_grouped', 'requested_by_regulator', 'study_topic_grouped'], level=0, errors='ignore')
        )

    ordered = pd.DataFrame()

    for variable in pre_order.index.get_level_values(0).unique():
        concat_df = pre_order.loc[[variable], :, :]

        correct_value_order = []
        if len(concat_df) == 2 and variable not in ['number_of_countries_grouped', 'updated_state']:
            correct_value_order = ['Yes', 'No']
        elif variable == 'age_population':
            correct_value_order = ['<18 years', '≥18 years', '<18 and ≥18 years']
        elif variable == 'funding_sources_grouped':
            correct_value_order = ['Commercial', 'Non-Commercial', 'Mixed', 'Unclear', 'No Funding']
        elif variable == 'number_of_subjects_grouped':
            correct_value_order = ['<100', '100-<500', '500-<1000', '1000-10000', '>10000']
        elif variable == 'requested_by_regulator':
            correct_value_order = ['Yes', 'No', 'Unspecified']
        elif len(concat_df) == 2 and variable == 'updated_state':
            correct_value_order = ['Ongoing', 'Finalised']

        if type == "logit" and sort_references_to_start:
            if not correct_value_order:
                correct_value_order = concat_df.index.get_level_values(1).to_list()
            is_reference = pre_order.loc[variable][pre_order.columns.difference(['N', 'n (%)'])].isna().all(axis='columns')
            reference_value = is_reference[is_reference == True].index.to_list()[0]
            reference_index = correct_value_order.index(reference_value)
            correct_value_order.insert(0, correct_value_order.pop(reference_index))

        if correct_value_order:
            concat_df = pre_order.loc[[variable], correct_value_order, :]

        ordered = pd.concat([
            ordered,
            concat_df
        ])
    
    ordered = ordered \
        .sort_index(
            level=0, 
            key=lambda x : x.map(dict(zip(variable_rename_and_reorder_map.keys(), range(len(variable_rename_and_reorder_map))))),
            sort_remaining=False
        ) \
        .rename(index=variable_rename_and_reorder_map, level=0)
    
    if type == 'frequency':
        ordered.columns = pd.MultiIndex.from_tuples(
            list(zip(
                [pre_order.columns.values[0], *['RMP requirement'] * 6], 
                ['', *pre_order.columns.values[1:]]
            ))
        )
    elif type == "logit":
        if multivariate_only:
            ordered = ordered.assign(
                **{
                    'OR multivariate': ordered['OR (95% CI) multivariate'].str.split(' ').str[0]
                },
                **{
                    '95% CI multivariate': ordered['OR (95% CI) multivariate'].str.split(r'[()]', regex=True).str[1]
                }
            )[['N', 'n (%)',
            'OR multivariate', '95% CI multivariate', 'P value multivariate']]
                
            ordered.columns = pd.MultiIndex.from_tuples(
                list(zip(
                    ['', '', *['Multivariate analysis']*3], 
                    ['N', 'n (%)', 'OR', '95% CI', 'P value']
                ))
            )
        else:
            ordered = ordered.assign(
                **{
                    f'OR {analysis_type}': ordered[f'OR (95% CI) {analysis_type}'].str.split(' ').str[0]
                    for analysis_type in ['univariate', 'multivariate']
                },
                **{
                    f'95% CI {analysis_type}': ordered[f'OR (95% CI) {analysis_type}'].str.split(r'[()]', regex=True).str[1]
                    for analysis_type in ['univariate', 'multivariate']
                }
            )[['N', 'n (%)', 'OR univariate', '95% CI univariate', 'P value univariate',
            'OR multivariate', '95% CI multivariate', 'P value multivariate']]

            ordered.columns = pd.MultiIndex.from_tuples(
                list(zip(
                    ['', '', *['Univariate analysis']*3, *['Multivariate analysis']*3], 
                    ['N', 'n (%)', *['OR', '95% CI', 'P value']*2]
                ))
            )
    
    return ordered

We will first format the frequency table:

In [77]:
[frequency_table_due_protocol, frequency_table_due_result] = pd.read_excel(
    INPUT_FOLDER / 'ema_rwd_final_statistics_tables_frequencies.xlsx', 
    sheet_name=['table_due_protocol', 'table_due_result'],
    index_col=(0, 1)
).values()

In [78]:
final_frequency_table_due_protocol = rename_and_reorder(frequency_table_due_protocol).drop(variable_rename_and_reorder_map['has_result'], level=0)
final_frequency_table_due_results = rename_and_reorder(frequency_table_due_result).drop(variable_rename_and_reorder_map['has_protocol'], level=0)

In [79]:
with pd.ExcelWriter('./ema_rwd_final_statistics_tables_frequencies_formated.xlsx') as writer:
    final_frequency_table_due_protocol.to_excel(
        writer,
        sheet_name='table_due_protocol'
    )
    final_frequency_table_due_results.to_excel(
        writer,
        sheet_name='table_due_result'
    )

Then the logit table:

In [9]:
[logit_table_due_protocol, logit_table_due_result] = pd.read_excel(
    INPUT_FOLDER / 'ema_rwd_final_statistics_tables_logit.xlsx',
    sheet_name=['table_due_protocol', 'table_due_result'],
    index_col=(0, 1)
).values()

In [10]:
final_logit_table_due_protocol = rename_and_reorder(logit_table_due_protocol, type='logit').fillna('')
final_logit_table_due_results = rename_and_reorder(logit_table_due_result, type='logit').fillna('')

In [11]:
final_logit_table = pd.merge(
    pd.concat({'Study protocol made public': final_logit_table_due_protocol}, axis='columns'),
    pd.concat({'Final study report made public': final_logit_table_due_results}, axis='columns'),
    left_index=True, right_index=True,
    how='left'
)

In [12]:
final_logit_table.to_excel('./ema_rwd_final_statistics_tables_logit_formated.xlsx', sheet_name='logit')

Then the logit table for the model without outliers:

In [13]:
logit_table_due_protocol_without_outliers = pd.read_excel(
    '../extra_analysis/ema_rwd_final_statistics_tables_logit_without_outliers_due_protocol.xlsx',
    index_col=(0, 1)
)

logit_table_due_result_without_outliers = pd.read_excel(
    '../extra_analysis/ema_rwd_final_statistics_tables_logit_without_outliers_due_result.xlsx',
    index_col=(0, 1)
)

In [14]:
final_logit_table_due_protocol_without_outliers = rename_and_reorder(logit_table_due_protocol_without_outliers, type='logit', multivariate_only=True).fillna('')
final_logit_table_due_results_without_outliers = rename_and_reorder(logit_table_due_result_without_outliers, type='logit', multivariate_only=True).fillna('')

In [15]:
final_logit_table_without_outliers = pd.merge(
    pd.concat({'Study protocol made public': final_logit_table_due_protocol_without_outliers}, axis='columns'),
    pd.concat({'Final study report made public': final_logit_table_due_results_without_outliers}, axis='columns'),
    left_index=True, right_index=True,
    how='left'
)

In [16]:
final_logit_table_without_outliers.to_excel('./ema_rwd_final_statistics_tables_logit_without_outliers_formated.xlsx', sheet_name='logit')

And finally the logit table for the sensitivity analysis:

In [17]:
logit_table_due_protocol_sensitivity = pd.read_excel(
    '../extra_analysis/ema_rwd_final_statistics_tables_logit_sensitivity_due_protocol.xlsx',
    index_col=(0, 1)
)

logit_table_due_result_logit_sensitivity = pd.read_excel(
    '../extra_analysis/ema_rwd_final_statistics_tables_logit_sensitivity_due_result.xlsx',
    index_col=(0, 1)
)

In [18]:
final_logit_table_due_protocol_sensitivity = rename_and_reorder(logit_table_due_protocol_sensitivity, type='logit', multivariate_only=True).fillna('')
final_logit_table_due_results_sensitivity = rename_and_reorder(logit_table_due_result_logit_sensitivity, type='logit', multivariate_only=True).fillna('')

In [19]:
final_logit_table_sensitivity = pd.merge(
    pd.concat({'Study protocol made public': final_logit_table_due_protocol_sensitivity}, axis='columns'),
    pd.concat({'Final study report made public': final_logit_table_due_results_sensitivity}, axis='columns'),
    left_index=True, right_index=True,
    how='left'
)

In [20]:
final_logit_table_sensitivity.to_excel('./ema_rwd_final_statistics_tables_logit_sensitivity_formated.xlsx', sheet_name='logit')

## Sponsor table

In [92]:
sponsors = pd.read_excel(
    '../../output/ema_rwd/ema_rwd_final_statistics_funding_all.xlsx', 
    index_col=0
)

sponsors.columns

Index(['Number Of Countries', 'Set Of Countries',
       'Number Of Studies With Updated State Finalised',
       'Number Of Studies With Updated State Ongoing',
       'Number Of Studies With Updated State Planned',
       'Number Of Studies With Updated State Nan',
       'Number Of Studies With Risk Management Plan Eu Rmp Category 1 (imposed As Condition Of Marketing Authorisation)',
       'Number Of Studies With Risk Management Plan Eu Rmp Category 2 (specific Obligation Of Marketing Authorisation)',
       'Number Of Studies With Risk Management Plan Eu Rmp Category 3 (required)',
       'Number Of Studies With Risk Management Plan Non-eu Rmp Only',
       'Number Of Studies With Risk Management Plan Not Applicable',
       'Number Of Studies With Risk Management Plan Nan',
       'Number Of Studies With Result', 'Number Of Studies With Protocol',
       'Number Of Studies With Due Protocol',
       'Number Of Studies With Due Protocol Has Protocol',
       'Number Of Studies Wit

In [93]:
sponsor_rename_column_map = {
    'Num Studies': 'N',
    'Number Of Studies With Due Protocol': 'Due Protocol',
    'Number Of Studies With Due Protocol Has Protocol': 'Due Protocol Has Protocol',
    'Number Of Studies With Due Result': 'Due Result',
    'Number Of Studies With Due Result Has Result': 'Due Result Has Result',
}

sponsor_rename_index_map = {
    np.nan: 'Sponsor unspecified',
    '$MarketingAuthorisationHolder': 'MAH not further specified',
    '$Name': 'Name of primary lead investigator specified',
    '$NoFunding': 'No funding',
    '$NotFound/UnclearValue': 'Specified sponsor unclear'
}

In [94]:
sponsors_formated = (
    sponsors[sponsor_rename_column_map.keys()]
    .rename_axis(index='Sponsor')
    .rename(
        index = lambda ax: re.sub(r'\s+\(.+\)$', '', sponsor_rename_index_map.get(ax, ax))
    ).rename(
        columns=sponsor_rename_column_map
    ).groupby(level=0).sum() # This is not needed anymore
)

sponsors_formated = (
    sponsors_formated.assign(
        **{
            f'Due {outcome} Has {outcome}': sponsors_formated[f'Due {outcome} Has {outcome}'].astype(str) + 
            ' (' + (100 * sponsors_formated[f'Due {outcome} Has {outcome}'] / sponsors_formated[f'Due {outcome}']).fillna(0).round(1).astype(str) + ')'
            for outcome in ['Protocol', 'Result']
        }
    )
    .reset_index() # Needed for correct sorting
    .sort_values(by=['N', 'Sponsor'], ascending=(False, True))
    .set_index('Sponsor')
)

sponsors_formated.columns = pd.MultiIndex.from_arrays([
    ['All Studies', 'Protocol due', 'Protocol due', 'Result due', 'Result due'],
    ['N', 'N', 'n', 'N', 'n']
])

sponsors_formated

All Studies Protocol due             Result due  \
                                    N            N           n          N   
Sponsor                                                                     
Pfizer                            176          142  119 (83.8)         91   
European Medicines Agency         157          155  100 (64.5)        135   
Amgen                             143          137  101 (73.7)        103   
Boehringer Ingelheim              122           93   34 (36.6)         51   
Specified sponsor unclear         110           77   28 (36.4)         46   
...                               ...          ...         ...        ...   
bene-Arzneimittel                   1            1     0 (0.0)          1   
bluebird bio                        1            1     0 (0.0)          0   
mibe                                1            1     0 (0.0)          0   
uniQure                             1            1     0 (0.0)          0   
ÚJV Řež                             1            1     0 (0.0)          0   

                                       
                                    n  
Sponsor                                
Pfizer                      83 (91.2)  
European Medicines Agency  106 (78.5)  
Amgen                       98 (95.1)  
Boehringer Ingelheim        29 (56.9)  
Specified sponsor unclear   20 (43.5)  
...                               ...  
bene-Arzneimittel             0 (0.0)  
bluebird bio                  0 (0.0)  
mibe                          0 (0.0)  
uniQure                       0 (0.0)  
ÚJV Řež                       0 (0.0)  

[638 rows x 5 columns]

In [95]:
sponsors_formated.to_excel('./ema_rwd_final_statistics_funding_all_formated.xlsx')